In [2]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
session = requests.session()

In [3]:
%store -r current_players
%store -r gameweeks

In [4]:
current_players

,team_id,player_id,team_name,round_score,player_name,gameweek
0,333932,169,All the Smallings,0,Schmeichel,26
1,297336,397,Childish Gbaminos,0,Noble,26
2,333932,220,All the Smallings,0,David Silva,26
3,336116,389,Stop Crying Lampard,0,Fabianski,26
4,300296,211,Ducks of Bethlehem,0,Agüero,26
...,...,...,...,...,...,...
85,299717,505,Obi-Wan Iwobi,0,James,26
86,339264,524,Neuer Gonna Give You,0,Sarr,26
87,300296,589,Ducks of Bethlehem,0,Minamino,26
88,299717,615,Obi-Wan Iwobi,0,Bergwijn,26


In [5]:
gameweek = 26
current_players["team_id"][0]

'333932'

In [6]:
# make list of all team ids in the league
team_ids = []
for i in range(len(current_players)):
    if current_players["team_id"][i] not in team_ids:
        team_ids.append(current_players["team_id"][i])

In [7]:
team_ids

['333932', '297336', '336116', '300296', '339264', '299717']

In [73]:
league_info = session.get("https://draft.premierleague.com/api/entry/333932/event/20")

In [74]:
parsed_player_gameweek = json.loads(league_info.text)

In [75]:
# investigate json structure
parsed_player_gameweek

{'picks': [{'element': 169,
   'position': 1,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 204,
   'position': 2,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 295,
   'position': 3,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 299,
   'position': 4,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 267,
   'position': 5,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 417,
   'position': 6,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 448,
   'position': 7,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 29,
   'position': 8,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 339,
   'position': 9,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier

In [66]:
# This is one player id in this team this week
for i in parsed_player_gameweek["subs"][i]["element_in"]
if 267 in parsed_player_gameweek["subs"][0]["element_in"]:
    print("yes")

TypeError: argument of type 'int' is not iterable

In [77]:
for el in parsed_player_gameweek["subs"]:
    if 295 == el["element_in"]:
        print("yes")

yes


In [61]:
parsed_player_gameweek["subs"][0]["element_out"]

220

In [12]:
for el in parsed_player_gameweek['picks']:
    print(el["element"])

169
204
122
299
267
448
29
417
130
167
12
149
220
203
186


In [13]:
all_players_df = pd.DataFrame()

In [14]:
def player_gameweek(team_id,weeks):
    # Will return the players in a team during all requested weeks. Expects as arguments a team id from fantasy draft premier league and the number of weeks you want data from.
    player_df = pd.DataFrame()

    for i in range(1,weeks+1):
        url = "https://draft.premierleague.com/api/entry/" + str(team_id)+ "/event/"+ str(i)
        player_gameweek = session.get(url)
        parsed_player_gameweek = json.loads(player_gameweek.text)
        players = []
        starting_11 = []
        gameweek = []
        team_ids = []
        subbed_in = []
        subbed_out = []
        temp_df = pd.DataFrame()
        for el in parsed_player_gameweek['picks']:
            team_ids.append(team_id)
            players.append(el["element"])
            gameweek.append(i)
            if players.append(el["position"]) <= 11:
                starting_11.append("yes")
            else:
                starting_11.append("no")
            for item in parsed_player_gameweek["subs"]:
                if el["element"] == item["element_in"]:
                    subbed_in.append("True")
                else:
                    
            for item in parsed_player_gameweek["subs"]:
                if el["element"] == item["element_out"]:
                    subbed_out.append("True")
            

        temp_df["team_id"] = team_ids
        temp_df["gameweek"] = gameweek 
        temp_df["player_id"] = players    
        player_df = player_df.append(temp_df)

    return player_df



    
    



In [15]:
player_gameweek(333932,25)

,team_id,gameweek,player_id
0,333932,1,149
1,333932,1,204
2,333932,1,122
3,333932,1,334
4,333932,1,346
...,...,...,...
10,333932,25,12
11,333932,25,149
12,333932,25,220
13,333932,25,203


In [16]:
for team in team_ids:
    all_players_df = all_players_df.append(player_gameweek(team,25))

In [17]:
# There are API endpoints with player scores for each week but each week is a different url. I will start by checking the layout of these pages

In [18]:
all_players_df

,team_id,gameweek,player_id
0,333932,1,149
1,333932,1,204
2,333932,1,122
3,333932,1,334
4,333932,1,346
...,...,...,...
10,299717,25,363
11,299717,25,14
12,299717,25,522
13,299717,25,505


In [19]:
# 6 teams in the league, 15 players per week, 25 weeks
6*15*25

2250

In [20]:

player_scores_gameweek1 = session.get("https://fantasy.premierleague.com/api/event/1/live")

In [21]:
player_scores_gameweek1_json = json.loads(player_scores_gameweek1.text)

In [22]:
player_scores_gameweek1_json


': 8,
    'in_dreamteam': False},
   'explain': [{'fixture': 2,
     'stats': [{'identifier': 'minutes', 'points': 1, 'value': 8},
      {'identifier': 'goals_scored', 'points': 4, 'value': 1},
      {'identifier': 'bonus', 'points': 3, 'value': 3}]}]},
  {'id': 303,
   'stats': {'minutes': 62,
    'goals_scored': 0,
    'assists': 0,
    'clean_sheets': 0,
    'goals_conceded': 1,
    'own_goals': 0,
    'penalties_saved': 0,
    'penalties_missed': 0,
    'yellow_cards': 0,
    'red_cards': 0,
    'saves': 0,
    'bonus': 0,
    'bps': 0,
    'influence': '2.0',
    'creativity': '5.3',
    'threat': '8.0',
    'ict_index': '1.5',
    'total_points': 2,
    'in_dreamteam': False},
   'explain': [{'fixture': 2,
     'stats': [{'identifier': 'minutes', 'points': 2, 'value': 62}]}]},
  {'id': 312,
   'stats': {'minutes': 0,
    'goals_scored': 0,
    'assists': 0,
    'clean_sheets': 0,
    'goals_conceded': 0,
    'own_goals': 0,
    'penalties_saved': 0,
    'penalties_missed': 0,
   

In [26]:
player_scores_gameweek1_json['elements'][0]

{'id': 14,
 'stats': {'minutes': 90,
  'goals_scored': 0,
  'assists': 0,
  'clean_sheets': 1,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 2,
  'bonus': 1,
  'bps': 27,
  'influence': '18.0',
  'creativity': '0.0',
  'threat': '0.0',
  'ict_index': '1.8',
  'total_points': 7,
  'in_dreamteam': False},
 'explain': [{'fixture': 10,
   'stats': [{'identifier': 'minutes', 'points': 2, 'value': 90},
    {'identifier': 'clean_sheets', 'points': 4, 'value': 1},
    {'identifier': 'bonus', 'points': 1, 'value': 1}]}]}

In [34]:
# I'll create a dictionary with the contents of each page so I don't end up sending a couple of thousand requests to their server

all_weeks_dict = {}
for i in range(1,26):
    #remember I'm getting 25 weeks worth of data change this as necessary for yourself
    url = "https://fantasy.premierleague.com/api/event/"+ str(i)+"/live"
    player_scores_gameweek = session.get(url)
    all_weeks_dict[i] = json.loads(player_scores_gameweek.text)
    




In [47]:
all_weeks_dict[1]["elements"][0]

{'id': 14,
 'stats': {'minutes': 90,
  'goals_scored': 0,
  'assists': 0,
  'clean_sheets': 1,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 2,
  'bonus': 1,
  'bps': 27,
  'influence': '18.0',
  'creativity': '0.0',
  'threat': '0.0',
  'ict_index': '1.8',
  'total_points': 7,
  'in_dreamteam': False},
 'explain': [{'fixture': 10,
   'stats': [{'identifier': 'minutes', 'points': 2, 'value': 90},
    {'identifier': 'clean_sheets', 'points': 4, 'value': 1},
    {'identifier': 'bonus', 'points': 1, 'value': 1}]}]}

In [120]:
all_players_df.reset_index(drop=True)

,team_id,gameweek,player_id
0,333932,1,149
1,333932,1,204
2,333932,1,122
3,333932,1,334
4,333932,1,346
...,...,...,...
2245,299717,25,363
2246,299717,25,14
2247,299717,25,522
2248,299717,25,505


In [43]:
id = []
total_points = []
gameweek = []
for j in all_weeks_dict:
        for x in all_weeks_dict[j]['elements']:
                print(x['stats']['total_points'])
                id.append(x["id"])
                total_points.append(x['stats']['total_points'])
                gameweek.append(j)



1
0
2
2
0
0
0
0
0
2
2
0
2
1
2
1
0
2
1
0
0
1
1
1
0
0
7
1
2
0
2
0
0
1
1
0
0
0
0
10
0
14
5
1
1
5
8
6
0
1
9
2
1
1
7
0
0
2
0
1
13
0
0
0
0
2
0
3
0
0
0
0
1
3
0
6
0
0
4
8
0
3
0
0
2
0
6
0
3
0
0
4
0
0
6
0
0
0
0
2
0
2
2
0
0
3
0
0
0
0
0
0
5
6
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
9
8
0
0
0
5
0
8
0
1
1
0
0
0
0
0
14
0
6
2
0
0
0
2
1
1
0
1
1
0
6
6
7
0
0
2
2
2
2
0
2
1
8
2
2
0
2
0
0
0
6
0
0
1
0
0
0
2
2
0
2
0
0
0
0
0
2
2
0
0
0
0
2
1
0
1
6
0
5
0
6
0
11
6
8
0
6
2
1
1
2
0
10
2
0
0
2
2
1
0
0
0
2
0
1
2
0
0
0
2
12
0
0
0
1
1
0
1
0
0
0
0
4
2
1
1
2
0
0
0
7
0
2
0
0
2
0
1
0
0
2
5
7
1
1
1
0
1
0
0
0
0
7
2
1
1
9
0
9
0
5
1
0
0
0
0
0
1
0
0
0
3
2
0
0
2
1
13
2
2
0
0
1
0
0
6
0
0
0
0
2
0
0
2
3
0
0
0
3
0
0
0
0
5
2
0
1
2
0
3
0
0
9
1
3
0
0
0
2
1
1
0
1
2
2
0
6
6
10
6
2
0
0
0
0
2
11
0
0
1
2
0
0
0
0
0
2
1
0
7
1
1
2
2
0
0
5
3
0
0
6
11
1
3
3
0
1
0
0
0
6
2
0
2
0
2
0
0
2
3
1
0
0
2
0
0
3
1
0
2
3
3
3
1
0
2
3
2
2
0
0
0
6
0
0
2
1
0
7
2
1
2
0
1
0
0
0
9
2
1
2
0
0
2
0
0
1
2
1
0
1
1
2
0
0
2
0
1
2
0
2
0
0


In [44]:
player_scores_df  = pd.DataFrame()

In [45]:
player_scores_df['player_id'] = id
player_scores_df['total_score'] = total_points
player_scores_df['gameweek'] = gameweek

In [46]:
player_scores_df

,player_id,total_score,gameweek
0,14,7,1
1,27,0,1
2,47,9,1
3,48,0,1
4,72,0,1
...,...,...,...
14106,581,0,25
14107,585,0,25
14108,592,0,25
14109,613,8,25


In [ ]:
# strictly I should really be keeping a table with all player scores, player information and team info as three separate tables in my database. To make my life easier later though I will combine all this into one table for my db. Hopefully I don't end up regretting this...

